统计分析每个子数据集问答对的token长度

In [10]:
import json
import os
from transformers import AutoTokenizer
import numpy as np

model_name_path = '/share/huggingface/Qwen-1_8B-Chat'
tok = AutoTokenizer.from_pretrained(model_name_path, eos_token='<|endoftext|>', pad_token='<|endoftext|>',unk_token='<|endoftext|>', trust_remote_code=True)

In [11]:
tok("你好，世界！"), tok("你好，世界！", return_tensors="pt")

({'input_ids': [108386, 3837, 99489, 6313], 'token_type_ids': [0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1]},
 {'input_ids': tensor([[108386,   3837,  99489,   6313]]), 'token_type_ids': tensor([[0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1]])})

In [12]:
tok.encode("你好，世界！"), tok.encode("你好，世界！", return_tensors="pt")

([108386, 3837, 99489, 6313], tensor([[108386,   3837,  99489,   6313]]))

In [16]:
root_dir = './'            
file_name_list = os.listdir(root_dir)
file_name_list.sort()

for file_name in file_name_list:
    if not file_name.endswith('.json'):
        continue
    print(file_name)
    path = os.path.join(root_dir, file_name)
    with open(path, 'r', encoding='utf-8') as f:
        data_list = json.load(f)

    prompt_token_len_list,target_token_len_list = [],[]
    for data in data_list:
        prompt_token_len_list.append(tok(data['prompt'], return_tensors='pt')['input_ids'].shape[1])
        target_token_len_list.append(tok(data['target_new'], return_tensors='pt')['input_ids'].shape[1])
    # 保留两位小数,四舍五入
    print('prompt token length:',round(np.mean(prompt_token_len_list),2))
    print('target token length:',round(np.mean(target_token_len_list),2))


type1_133.json
prompt token length: 22.31
target token length: 12.38
type2_80.json
prompt token length: 15.16
target token length: 16.85
type3_40.json
prompt token length: 8.8
target token length: 39.0
type4_50.json
prompt token length: 9.86
target token length: 8.68
type5_70.json
prompt token length: 24.56
target token length: 2.7
type6_50.json
prompt token length: 8.28
target token length: 8.66
type7_277.json
prompt token length: 18.22
target token length: 195.69


In [ ]:
from transformers import AutoTokenizer

# Load the tokenizer
model_path = '/share/huggingface/'
model_name = 'Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_path + model_name, trust_remote_code=True)

# Print some tokens from the tokenizer's vocabulary to identify a suitable PAD/EOS token
for i, token in enumerate(tokenizer.get_vocab()):
    if i < 10:  # Print the first 10 tokens as a sample
        print(f"Token ID: {i}, Token: {tokenizer.convert_ids_to_tokens(i)}")


In [ ]:
len(tokenizer.get_vocab())

In [ ]:
list(tokenizer.get_vocab().keys())[-1]

In [ ]:
tokenizer.convert_ids_to_tokens(151643)

In [ ]:
from transformers import AutoTokenizer

model_path = '/share/huggingface/'
model_name = 'Qwen-1_8B-Chat'
tokenizer = AutoTokenizer.from_pretrained(model_path + model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token = '<|endoftext|>'
tokenizer.pad_token_id = tokenizer.eos_token_id = 151643
prompt_tokens = tokenizer.batch_encode_plus(prompt_list, padding=True, return_tensors='pt')['input_ids']
# ValueError: Asking to pad but the tokenizer does not have a padding token. Please select a token to use as `pad_token` `(tokenizer.pad_token = tokenizer.eos_token e.g.)` or add a new pad token via `tokenizer.add_special_tokens({'pad_token': '[PAD]'})`.

In [ ]:
prompt_tokens.shape

In [ ]:
def tokenize_prompt_list(prompt_list):
    token_list, token_length_list = [], []
    for prompt in prompt_list:
        token_list.append(tokenizer(prompt, padding=False)['input_ids'])
        token_length_list.append(len(token_list[-1]))
    return token_list, token_length_list

token_list, token_length_list = tokenize_prompt_list(prompt_list)
sum(token_length_list) / len(token_length_list)